In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata

In [3]:
class DataLoad():
    def __init__(self, inpath,):
        self.inpath = inpath

    def load(self):      
        with open(self.inpath, 'r') as f:
            data = pd.DataFrame().from_dict(json.load(f)).T.reset_index(names="question_number")
            f.close()
        return data

In [4]:
inpath = '../train/questions.json'
df_train = DataLoad(inpath,).load()
df_train['flag'] = 'train'
inpath = '../test/questions.json'
df_test = DataLoad(inpath,).load()
df_test['flag'] = 'test'
inpath = '../valid/questions.json'
df_valid = DataLoad(inpath,).load()
df_valid['flag'] = 'valid'

data = pd.concat([df_train, df_test, df_valid], axis=0)   

In [11]:
import xml.etree.ElementTree as ET
tree = ET.parse('../../label.xml')
root = tree.getroot()
taxonomy = []
for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

tree = ET.parse('../../label2.xml')
root = tree.getroot()

for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

tree = ET.parse('../../label3.xml')
root = tree.getroot()
for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

taxonomy.append(['NonControllingInterestsIFRSMember','非支配持分'])
taxonomy.append(['OutsideDirectorsAuditAndSupervisoryCommitteeMember','社外取締役'])
taxonomy.append(['Ordinary share','社外監査役'])


df = pd.DataFrame(taxonomy, columns = ['en_label', 'ja_label']).drop_duplicates()
df['en_label'] = df['en_label'].apply(lambda x: unicodedata.normalize("NFKC", x).replace('label_', ''))

In [12]:
df.query('"OrdinaryShareMember" in en_label')

,en_label,ja_label
12155,OrdinaryShareMember,普通株式


In [13]:
ja_question = []
for index, row in data.iterrows():
    row['ja_question'] = unicodedata.normalize("NFKC", row['question'])
    en_label = re.findall(r'[a-zA-Z]+', row['question'])
    for label in en_label:
        if label in df['en_label'].values:
            ja_label = df.query('en_label == @label')['ja_label'].values[0]
            row['ja_question'] = row['ja_question'].replace(label, ja_label)
        row['ja_question'] = row['ja_question'].replace('(IFRS)', '')
    ja_question.append(row['ja_question'])
data['ja_question'] = ja_question

In [14]:
def process_query(query):
    #質問文を整形する
    if re.search(r"「.*」",query):
        contents = re.search(r"「(.*)」",query).group(0)[1:-1].split("、")
        return query.replace(re.search(r"「.*」",query).group(0), 'の'.join(contents[::-1]))
data['ja_question'] = data['ja_question'].apply(process_query) 

In [15]:
for index, row in data.iterrows():
    try:
        if re.search(r"^(.*?の)", row['ja_question']):
            text = re.sub(r"^(.*?の)", '', row['ja_question'])
            data.loc[index, 'tmp_question'] = text
    except:
        print(row['ja_question'])
        

In [16]:
#タクソノミの英語ラベルと「」で囲まれた部分の抽出
def extract_words(text):
    words = []
    for word in re.findall(r"「.*」", text):
        words.append(word[1:-1])

    words.extend(re.findall(r'[a-zA-Z]+', text))
    
    return words

data['words'] = data['question'].apply(extract_words)

In [17]:
data.head()

,question_number,question,doc_id,table_id,cell_id,value,flag,ja_question,tmp_question,words
0,question_train3,株式会社アインホールディングスの2021年における「１株当たり当期純利益又は当期純損失（△）...,S100RHMW,S100RHMW-0101010-tab2,S100RHMW-0101010-tab2-r10c5,189.04,train,株式会社アインホールディングスの2021年における経営指標等の1株当たり当期純利益又は当期純...,2021年における経営指標等の自己資本利益率は?,[１株当たり当期純利益又は当期純損失（△）、経営指標等]
1,question_train4,株式会社アインホールディングスの2023年における「１株当たり当期純利益又は当期純損失（△）...,S100RHMW,S100RHMW-0101010-tab2,S100RHMW-0101010-tab2-r10c7,262.87,train,株式会社アインホールディングスの2023年における経営指標等の1株当たり当期純利益又は当期純...,2023年における経営指標等の自己資本利益率は?,[１株当たり当期純利益又は当期純損失（△）、経営指標等]
2,question_train5,株式会社アインホールディングスの2022年時点のNonConsolidatedMemberに...,S100RHMW,S100RHMW-0101010-tab3,S100RHMW-0101010-tab3-r8c6,78237000000,train,株式会社アインホールディングスの2022年時点の非連結又は個別における経営指標等の純資産額は?,2023年時点の普通株式における所有株式数の割合(%)-金融商品取引業者は?,"[純資産額、経営指標等, NonConsolidatedMember]"
3,question_train9,株式会社アインホールディングスの2023年時点のNo3MajorShareholdersMe...,S100RHMW,S100RHMW-0104010-tab28,S100RHMW-0104010-tab28-r5c1,株式会社セブン&amp;アイ・ホールディングス,train,株式会社アインホールディングスの2023年時点のNo3大株主における大株主の状況の氏名又は名称は?,2023年時点のNo5大株主における大株主の状況の氏名又は名称は?,"[氏名又は名称、大株主の状況, No, MajorShareholdersMember]"
4,question_train12,株式会社アインホールディングスの2023年時点のSharesWithRestrictedVo...,S100RHMW,S100RHMW-0104010-tab29,S100RHMW-0104010-tab29-r4c5,－,train,株式会社アインホールディングスの2023年時点の議決権制限株式（自己株式等）における議決権の...,2023年時点の議決権制限株式（その他）における議決権の状況の発行済株式の内容は?,"[内容、発行済株式、議決権の状況, SharesWithRestrictedVotingRi..."


In [18]:
data.to_csv('../data.csv', index=False)